In [ ]:
import json
import os
from parse_annotation import parse_annotation
from somajo import SoMaJo as smj
import jellyfish
import math
import trankit

tokenizer = smj(language="en_PTB")
BASE_DATA_PATH = "/home/burak/Desktop/thesis/code/IM/twitter_data"
THREADS_PATH = BASE_DATA_PATH + "/raw"
TOKENIZED_THREADS_PATH = BASE_DATA_PATH + "/raw/tokenized"
ANN_PATH = BASE_DATA_PATH + "/ann/raw"
FIXED_ANN_PATH = BASE_DATA_PATH + "/ann/fixed_raw"
file_name = "514_947136019320651777.branch1.txt.username_text_tabseparated"


In [ ]:
files = os.listdir(THREADS_PATH)

readable_annotations = dict()

for file in files:
    try:
        if file[-12:] == 'tabseparated':
            readable_annotations[file] = list()
            for annotation in parse_annotation(
                THREADS_PATH + '/' + file,
                FIXED_ANN_PATH + '/' + file,
                extract_16=False
            ):
                readable_annotations[file].append(annotation)
    except Exception as inst:
        print(file, type(inst))

In [ ]:
conll_format_dict = dict()
for file_name in readable_annotations.keys():
    file_annotations = readable_annotations[file_name]
    conll_format_dict[file_name] = []
    
    token_index = 0
    tokens = []
    with open(TOKENIZED_THREADS_PATH + '/' + file_name) as f:
        text = f.read()
        tokenized_thread = json.loads(text)
        for sentence_index, sentence in enumerate(tokenized_thread['sentences']):
            for token_sentence_index, token in enumerate(sentence['tokens']):
                tokens.append([
                    token['characterOffsetBegin'],
                    token['characterOffsetEnd'],
                    len(tokens),
                    sentence_index,
                    token_sentence_index,
                    token['surface']
                ])
    
    for annotation_index, annotation in enumerate(file_annotations):
        conll_annotation = dict()
        
        index_cursor = 0
        
        # arg1
        arg1 = dict()
        arg1['TokenList'] = []
        arg1['RawText'] = []
        arg1['CharacterSpanList'] = []
        
        if annotation['arg1_range'] != '':
            for i, arg1_range_part in enumerate(annotation['arg1_range'].split(';')):
                arg1_range_part_start = int(arg1_range_part.split('..')[0])
                arg1_range_part_end = int(arg1_range_part.split('..')[1])
                arg1['CharacterSpanList'].append([int(range_offset) for range_offset in arg1_range_part.split('..')])
                arg1_range_text = annotation['arg1'][i]
                arg1['RawText'].append(arg1_range_text)
                
                # find token indexes for arg1
                
                start_index = None
                end_index = None
                
                #find start
                for i in range(len(tokens)):
                    if arg1_range_part_start == tokens[i][0]:
                        start_index = i 
                        break
                    if arg1_range_part_start > tokens[i][0] and arg1_range_part_start < tokens[i][1]:
                        start_index = i
                        print(tokens[i])
                        print(arg1_range_part_start)
                        print('arg1 start not equal')
                        break
                    if arg1_range_part_start == tokens[i][1]:
                        start_index = i + 1
                        print(tokens[i + 1])
                        print(tokens[i])
                        print(arg1_range_part_start)
                        print('arg1 start exceeded')
                        print(file_name)
                        break
                        
                if start_index == None:
                    print(arg1_range_part)
                    print(tokens)
                    print('arg1 start in', file_name)
                    continue
                    
                
                #find end
                for i in range(len(tokens)):
                    if arg1_range_part_end == tokens[i][1]:
                        end_index = i 
                        break
                    if arg1_range_part_end > tokens[i][0] and arg1_range_part_end < tokens[i][1]:
                        end_index = i
                        print(tokens[i])
                        print(arg1_range_part_end)
                        print('arg1 end not equal')
                        break
                    if arg1_range_part_end == tokens[i][0]:
                        end_index = i - 1
                        print(tokens[i - 1])
                        print(tokens[i])
                        print(arg1_range_part_end)
                        print('arg1 end exceeded 1')
                        break
                    if arg1_range_part_end < tokens[i][0] and arg1_range_part_end < tokens[i][1]:
                        end_index = i - 1
                        print(tokens[i - 1])
                        print(tokens[i])
                        print(arg1_range_part_end)
                        print('arg1 end exceeded 2')
                        print(file_name)
                        break

                if end_index == None:
                    print(arg1_range_part)
                    print(tokens)
                    print('arg1 end error in', file_name)
                    continue
                
                for t in tokens[start_index : end_index + 1]:
                    arg1['TokenList'].append(t[0:5])
            
        
        
        # arg2
        arg2 = dict()
        arg2['TokenList'] = []
        arg2['RawText'] = []
        arg2['CharacterSpanList'] = []
        
        if annotation['arg2_range'] != '':
            for i, arg2_range_part in enumerate(annotation['arg2_range'].split(';')):
                arg2_range_part_start = int(arg2_range_part.split('..')[0])
                arg2_range_part_end = int(arg2_range_part.split('..')[1])
                arg2['CharacterSpanList'].append([int(range_offset) for range_offset in arg2_range_part.split('..')])
                arg2_range_text = annotation['arg2'][i]
                arg2['RawText'].append(arg2_range_text)
                
                # find token indexes for arg1
                
                start_index = None
                end_index = None
                
                #find start
                for i in range(len(tokens)):
                    if arg2_range_part_start == tokens[i][0]:
                        start_index = i
                        break
                    if arg2_range_part_start > tokens[i][0] and arg2_range_part_start < tokens[i][1]:
                        start_index = i
                        print(tokens[i])
                        print(arg2_range_part_start)
                        print('arg2 start not equal')
                        break
                    if arg2_range_part_start == tokens[i][1]:
                        start_index = i + 1
                        print(tokens[i + 1])
                        print(tokens[i])
                        print(arg2_range_part_start)
                        print('arg2 start exceeded')
                        print(file_name)
                        break
                
                if start_index == None:
                    print(arg2_range_part)
                    print(tokens)
                    print('arg2 start error in', file_name)
                    continue
                
                #find end
                for i in range(len(tokens)):
                    if arg2_range_part_end == tokens[i][1]:
                        end_index = i 
                        break
                    if arg2_range_part_end > tokens[i][0] and arg2_range_part_end < tokens[i][1]:
                        end_index = i
                        print(tokens[i])
                        print(arg2_range_part_end)
                        print('arg2 end not equal')
                        break
                    if arg2_range_part_end == tokens[i][0]:
                        end_index = i - 1
                        print(tokens[i - 1])
                        print(tokens[i])
                        print(arg2_range_part_end)
                        print('arg2 end exceeded 1')
                        break
                    if arg2_range_part_end < tokens[i][0] and arg2_range_part_end < tokens[i][1]:
                        end_index = i - 1
                        print(tokens[i - 1])
                        print(tokens[i])
                        print(arg2_range_part_end)
                        print('arg2 end exceeded 2')
                        print(file_name)
                        break

                if end_index == None:
                    print(arg2_range_part)
                    print(tokens)
                    print('arg2 end error in', file_name)
                    continue
                
                for t in tokens[start_index : end_index + 1]:
                    arg2['TokenList'].append(t[0:5])
        
        # connective
        connective = dict()
        connective['TokenList'] = []
        connective['RawText'] = []
        connective['CharacterSpanList'] = []
        
        if '..' in annotation['connective_range']:
            for i, connective_range_part in enumerate(annotation['connective_range'].split(';')):
                connective_range_part_start = int(connective_range_part.split('..')[0])
                connective_range_part_end = int(connective_range_part.split('..')[1])
                connective['CharacterSpanList'].append([int(range_offset) for range_offset in connective_range_part.split('..')])
                connective_range_text = annotation['connective'][i]
                

                if connective_range_text == '&amp':
                    connective_range_text = '&amp;'
                
                connective['RawText'].append(connective_range_text)
                
                # find token indexes for connective
                
                start_index = None
                end_index = None
                
                #find start
                for i in range(len(tokens)):
                    if connective_range_part_start == tokens[i][0]:
                        start_index = i 
                        break
                    if connective_range_part_start > tokens[i][0] and connective_range_part_start < tokens[i][1]:
                        start_index = i
                        print(tokens[i])
                        print(connective_range_part_start)
                        print('connective start not equal')
                        break
                    if connective_range_part_start == tokens[i][1]:
                        start_index = i + 1
                        print(tokens[i + 1])
                        print(tokens[i])
                        print(connective_range_part_start)
                        print('connective start exceeded')
                        print(file_name)
                        break
                        
                if start_index == None:
                    print(connective_range_part)
                    print('connective start error in', file_name)
                    continue
                
                #find end
                for i in range(len(tokens)):
                    if connective_range_part_end == tokens[i][1]:
                        end_index = i 
                        break
                    if connective_range_part_end > tokens[i][0] and connective_range_part_end < tokens[i][1]:
                        end_index = i
                        print(tokens[i])
                        print(connective_range_part_end)
                        print('connective end not equal')
                        break
                    if connective_range_part_end == tokens[i][0]:
                        end_index = i - 1
                        print(tokens[i - 1])
                        print(tokens[i])
                        print(connective_range_part_end)
                        print('connective end exceeded 1')
                        break
                    if connective_range_part_end < tokens[i][0] and connective_range_part_end < tokens[i][1]:
                        end_index = i - 1
                        print(tokens[i - 1])
                        print(tokens[i])
                        print(tokens)
                        print(connective_range_part_end)
                        print('connective end exceeded 2')
                        print(file_name)
                        break

                if end_index == None:
                    print('connective error in', file_name)
                    continue
                
                for t in tokens[start_index : end_index + 1]:
                    connective['TokenList'].append(t[0:5])
        
        relation = {
            'Arg1': arg1,
            'Arg2': arg2,
            'Connective': connective,
            'Sense': [annotation['sense']],
            'Type': annotation['type'],
            'DocID': file_name,
            'ID': file_name[0:3] + str(annotation_index)
        }
        conll_format_dict[file_name].append(relation)

json_string = json.dumps(conll_format_dict)
with open('./conll_relations_raw_fixed_spans.json', 'w') as outfile:
    outfile.write(json_string)
    

In [ ]:

# find token indexes for arg2
for index_window in range(len(tokens) - len(arg1_tokens) + 1):
    distance = jellyfish.levenshtein_distance(
        ' '.join(arg1_tokens),
        ' '.join(tokens_text[index_window : index_window + len(arg1_tokens)])
    )
    if distance <= 2:
        arg1_token_index = index_window
        break

In [ ]:
best_distance = math.inf
best_option = None
for index_window in range(len(tokens) - len(arg1_tokens) + 1):
    distance = jellyfish.levenshtein_distance(
        ' '.join(arg1_tokens),
        ' '.join(tokens_text[index_window : index_window + len(arg1_tokens)])
    )
    if (distance < best_distance):
        best_distance = distance
        arg1_token_index = index_window

    if distance <= 2:
        arg1_token_index = index_window
        break